### Problem Statement : Hospital Patient Data Analysis
#### Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.
Tasks:
1.	Load the patient dataset and show summary with info().
2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
4.	Use groupby to find total bill amount per department.
5.	Remove duplicate patient records based on PatientID.
6.	Fill missing BillAmount values with the mean bill amount.
7.	Merge the billing dataset with patient dataset on PatientID.
8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

Expected Outcome:
 - Final cleaned dataset with accurate billing info.
 - All missing values handled, merged dataset across PatientID.
 - Ability to perform further analytics on department-wise revenue or doctor performance.



In [67]:
# 1.	Load the patient dataset and show summary with info().
import pandas as pd
patient = pd.read_csv('Patient_Data.csv')
patient_data = patient
print(patient_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None


In [68]:
# 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
patient_df = patient_data[['PatientID', 'Department', 'Doctor', 'BillAmount']]
print(patient_df.head())

   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN


In [69]:
# 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
patient_data = patient_data.drop(columns=['ReceptionistID', 'CheckInTime'])
print(patient_data.head())

   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


In [70]:
# 4.	Use groupby to find total bill amount per department.
dept_billing = patient_data.groupby('Department')['BillAmount'].sum().reset_index()
print(dept_billing)

    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [71]:
# 5.	Remove duplicate patient records based on PatientID.
patient_data = patient_data.drop_duplicates(subset=['PatientID'])
print(patient_data.head())

   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


In [72]:
# 6.	Fill missing BillAmount values with the mean bill amount.
mean_bill = patient_data['BillAmount'].mean()
patient_data['BillAmount'] = patient_data['BillAmount'].fillna(round(mean_bill, 2))
print(patient_data.head())

   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith     5000.00
1        102      Bob    Neurology   Dr. John     6233.33
2        103  Charlie  Orthopedics    Dr. Lee     7500.00
3        104    David   Cardiology  Dr. Smith     6200.00
4        105      Eva  Dermatology   Dr. Rose     6233.33


In [73]:
# 7.	Merge the billing dataset with patient dataset on PatientID.
billing_data = pd.read_csv('Billing_Data.csv')
merged_data = pd.merge(patient_data, billing_data, on='PatientID', how='inner')
print(merged_data.head())

   PatientID     Name   Department     Doctor  BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith     5000.00              2000   
1        102      Bob    Neurology   Dr. John     6233.33              1500   
2        103  Charlie  Orthopedics    Dr. Lee     7500.00              2500   
3        104    David   Cardiology  Dr. Smith     6200.00              3000   
4        105      Eva  Dermatology   Dr. Rose     6233.33              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  


In [74]:
# 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
new_patients = pd.DataFrame({
    'PatientID': [101, 102],
    'Name': ['Hari', 'Shyam'],
    'Department': ['Cardiology', 'Neurology'],
    'BillAmount': [8900.00, 10250.00],
    'Doctor': ['Dr. Smith', 'Dr. Jones'],
    'CheckInTime': ['2024-10-01 10:00', '2024-10-01 11:00'],
    'ReceptionistID': [5, 6],
})
all_patients = pd.concat([patient, new_patients], axis=0, ignore_index=True)
print(all_patients)

   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
1        102      Bob    Neurology   Dr. John         NaN               2   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
3        104    David   Cardiology  Dr. Smith      6200.0               3   
4        105      Eva  Dermatology   Dr. Rose         NaN               2   
5        101    Alice   Cardiology  Dr. Smith      5000.0               1   
6        101     Hari   Cardiology  Dr. Smith      8900.0               5   
7        102    Shyam    Neurology  Dr. Jones     10250.0               6   

        CheckInTime  
0  2023-01-10 09:00  
1  2023-01-11 10:30  
2  2023-01-12 11:00  
3  2023-01-13 12:00  
4  2023-01-14 08:45  
5  2023-01-10 09:00  
6  2024-10-01 10:00  
7  2024-10-01 11:00  


In [75]:
# 9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
insurance = [3000.00, 5000.00, 4000.00, 6000.00, 4500.00, 2500.00, 3000.00, 4000.00]
billing_info = pd.DataFrame({
    'InsuranceCovered': insurance,
    'FinalAmount': [all_patients['BillAmount'][i] - insurance[i] for i in range(len(insurance))]
})
all_patients = pd.concat([all_patients, billing_info], axis=1)
print(all_patients)

   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
1        102      Bob    Neurology   Dr. John         NaN               2   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
3        104    David   Cardiology  Dr. Smith      6200.0               3   
4        105      Eva  Dermatology   Dr. Rose         NaN               2   
5        101    Alice   Cardiology  Dr. Smith      5000.0               1   
6        101     Hari   Cardiology  Dr. Smith      8900.0               5   
7        102    Shyam    Neurology  Dr. Jones     10250.0               6   

        CheckInTime  InsuranceCovered  FinalAmount  
0  2023-01-10 09:00            3000.0       2000.0  
1  2023-01-11 10:30            5000.0          NaN  
2  2023-01-12 11:00            4000.0       3500.0  
3  2023-01-13 12:00            6000.0        200.0  
4  2023-01-14 08:45            4500.0    

In [76]:
print(all_patients[['InsuranceCovered', 'FinalAmount']])

   InsuranceCovered  FinalAmount
0            3000.0       2000.0
1            5000.0          NaN
2            4000.0       3500.0
3            6000.0        200.0
4            4500.0          NaN
5            2500.0       2500.0
6            3000.0       5900.0
7            4000.0       6250.0
